In [44]:
%matplotlib inline
import math as m
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
from SAMPEX_functions import read_counts as read
from SAMPEX_functions import mb_finder, mb_magnitude, iso_calculator, OrbAtt_augment, OrbAtt_augment_loop
import pandas as pd
import os
import pathlib
import string

In [45]:
# defining file
year = '1993'
start_day = '1'
end_day = '30'
Output_Plots = False

In [46]:
# code is specific for year 1993
OrbAtt_names = np.array(['OrbAtt_secofyear_1993001-1993012.txt', 'OrbAtt_secofyear_1993013-1993039.txt', 'OrbAtt_secofyear_1993040-1993066.txt', \
                         'OrbAtt_secofyear_1993067-1993093.txt', 'OrbAtt_secofyear_1993094-1993120.txt', 'OrbAtt_secofyear_1993121-1993147.txt', \
                         'OrbAtt_secofyear_1993148-1993174.txt', 'OrbAtt_secofyear_1993175-1993201.txt', 'OrbAtt_secofyear_1993202-1993228.txt', \
                         'OrbAtt_secofyear_1993229-1993255.txt', 'OrbAtt_secofyear_1993256-1993282.txt', 'OrbAtt_secofyear_1993283-1993309.txt', \
                         'OrbAtt_secofyear_1993310-1993336.txt', 'OrbAtt_secofyear_1993337-1993363.txt', 'OrbAtt_secofyear_1993364-1993365.txt'])

In [47]:
# define mag indices data
directory = 'E:\SAMPEX_Data\\'
file = 'mag_indices_1993.txt'
mag_indices = pd.read_csv(directory + file, header = None, delimiter='\s+')

# define day, hr, min, AE, and Dst
ind_day = np.array(mag_indices[1])
ind_hr = np.array(mag_indices[2])
ind_min = np.array(mag_indices[3])
AE = np.array(mag_indices[4])
Dst = np.array(mag_indices[5])

In [48]:
########### INITIALIZING MB & BG ISO vs PARAM HISTOGRAMS ############
max_iso = 1.0
max_AE = int(m.ceil(np.max(AE)/100)*100)
bin_num = 25

iso_bins = np.linspace(0, max_iso, num=bin_num+1)
AE_bins = np.linspace(0, max_AE, num=bin_num+1)

###### MICROBURST ISO vs PARAMETERS #######
# Create empty microburst histograms
H_IvAE_MB, _, _ = np.histogram2d(np.array([]), np.array([]), bins=[AE_bins, iso_bins])

###### BACKGROUND ISO vs PARAMETERS #######
# Create empty background histograms
H_IvAE_BG, _, _ = np.histogram2d(np.array([]), np.array([]), bins=[AE_bins, iso_bins])

In [49]:
for day_of_year in np.arange(int(start_day), int(end_day)+1):
    # name the day
    if len(str(day_of_year)) == 1:
        file = 'hhrr' + year + '00' + str(day_of_year) + '.txt'
    elif len(str(day_of_year)) == 2:
        file = 'hhrr' + year + '0' + str(day_of_year) + '.txt'
    elif len(str(day_of_year)) == 3:
        file = 'hhrr' + year + str(day_of_year) + '.txt'

    # reading in electron counts files:
    t_electrons, r1, r2, r3, r4 = read(year + '_data\\' + file)

    # reading in orbit/attitude data file
    OrbAtt_mask = []
    for i in np.arange(len(OrbAtt_names)):
        OrbAtt_mask.append(day_of_year >= int(OrbAtt_names[i][21:24]) and day_of_year <= int(OrbAtt_names[i][29:32]))
        
    OA_file = OrbAtt_names[OrbAtt_mask][0]
    
    directory = 'E:\SAMPEX_Data\\' + year + '_data\\'
    OrbAtt_file = pd.read_csv(directory + OA_file, names = ['day', 'hr', 'min', 'sec', 'GEO_Radius', 'GEO_Long', 'GEO_Lat', 'Altitude', 
                                                            'L_Shell', 'MLT', 'SAA_Flag', 'Pitch', 'zenith', 'azimuth', 'Att_Flag'], sep = '\s+', header = 70)
    
    # Augment OrbAtt data to fit counts data
    t_OrbAtt, LS_OrbAtt, MLT_OrbAtt, P_OrbAtt, Lat_OrbAtt, Long_OrbAtt, R_OrbAtt, t_electrons, r1, r2, r3, r4 = OrbAtt_augment_loop(t_electrons, r1, r2, r3, r4, OrbAtt_file, day_of_year)


    ############## mask for spatial/temporal regions ##############
    
    mask = (np.abs(90 - P_OrbAtt) >= 50) & (LS_OrbAtt >= 3) & (LS_OrbAtt <= 8)
    t_OrbAtt = t_OrbAtt[mask]; LS_OrbAtt = LS_OrbAtt[mask]; MLT_OrbAtt = MLT_OrbAtt[mask];
    P_OrbAtt = P_OrbAtt[mask]; Lat_OrbAtt = Lat_OrbAtt[mask]; Long_OrbAtt = Long_OrbAtt[mask];
    R_OrbAtt = R_OrbAtt[mask]; t_electrons = t_electrons[mask]; r1 = r1[mask]; r2 = r2[mask]; r3 = r3[mask]; r4 = r4[mask]

    ###############################################################
    
    
    # find microburst times and N_100, SSD1, SSD4 counts using algorithm
    t_microburst, N_100_microburst, r1_microburst, r4_microburst, mb_index, MB_mask, N_100, A_500 = mb_finder(t_electrons, r1, r2, r3, r4)

    # create background counts mask
    BG_mask = ~MB_mask

    # calculation of the isotropy indices of electron counts
    iso_indices_MB = iso_calculator(r1[MB_mask], r4[MB_mask])
    iso_indices_BG = iso_calculator(r1[BG_mask], r4[BG_mask])

    # find the OrbAtt and magnetic index microburst and backgound parameters
    if len(mb_index) > 0:
        t_OrbAtt_mb = t_OrbAtt[mb_index]; LS_OrbAtt_mb = LS_OrbAtt[mb_index]; MLT_OrbAtt_mb = MLT_OrbAtt[mb_index];
        P_OrbAtt_mb = P_OrbAtt[mb_index]; Lat_OrbAtt_mb = Lat_OrbAtt[mb_index]; Long_OrbAtt_mb = Long_OrbAtt[mb_index];
        R_OrbAtt_mb = R_OrbAtt[mb_index];
        
        AE_day_MB = np.array([])
        for t in t_microburst:
            hr = t/3600
            if t % 3600 != 0:
                hr_round = m.ceil(hr)-1
                minute = t/60 - hr_round*60
                if minute % 5 != 0:
                    five_min = (m.ceil(minute/5)-1)*5
                elif minute % 5 == 0:
                    five_min = int(minute)
            elif t % 3600 == 0:
                hr_round = int(hr)
                minute = 0
                five_min = 0
            
            if hr_round == 24:
                day_of_year_new = day_of_year + 1
                hr_round = 0
                AE_MB = AE[(ind_day == day_of_year_new) & (ind_hr == hr_round) & (ind_min == five_min)]    
        
            elif hr_round != 24:
                AE_MB = AE[(ind_day == day_of_year) & (ind_hr == hr_round) & (ind_min == five_min)]
            
            AE_day_MB = np.append(AE_day_MB, AE_MB)

    t_BG = t_electrons[BG_mask]
    AE_day_BG = np.array([])
    for t in t_BG:
        hr = t/3600
        if t % 3600 != 0:
            hr_round = m.ceil(hr)-1
            minute = t/60 - hr_round*60
            if minute % 5 != 0:
                five_min = (m.ceil(minute/5)-1)*5
            elif minute % 5 == 0:
                five_min = int(minute)
            
        elif t % 3600 == 0:
            hr_round = int(hr)
            minute = 0
            five_min = 0
            
        if hr_round == 24:
            day_of_year_new = day_of_year + 1
            hr_round = 0
            AE_BG = AE[(ind_day == day_of_year_new) & (ind_hr == hr_round) & (ind_min == five_min)]    
        
        elif hr_round != 24:
            AE_BG = AE[(ind_day == day_of_year) & (ind_hr == hr_round) & (ind_min == five_min)]
        
        AE_day_BG = np.append(AE_day_BG, AE_BG)
    
    ###### MICROBURST ISO vs PARAMETERS #######
    # Create histograms for day #
    if len(mb_index) > 0:
        H_IvAE_MB_day, _, _ = np.histogram2d(AE_day_MB, iso_indices_MB, bins=[AE_bins, iso_bins])
    
        # Add histograms for day # to cumulative histograms
        H_IvAE_MB += H_IvAE_MB_day
    
    ###### BACKGROUND ISO vs PARAMETERS #######
    # Create histograms for day #
    H_IvAE_BG_day, _, _ = np.histogram2d(AE_day_BG, iso_indices_BG, bins=[AE_bins, iso_bins])
    
    # Add histograms for day # to cumulative histograms
    H_IvAE_BG += H_IvAE_BG_day

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10,10))
plt.pcolormesh(AE_bins, iso_bins, H_IvAE_MB.T, vmin=0)